In [1]:
import codecs
import os
import collections
import _pickle as cPickle
#from six.moves import cPickle
import numpy as np


class TextLoader():
    def __init__(self, data_dir, batch_size, seq_length, encoding='utf-8'):
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.seq_length = seq_length
        self.encoding = encoding

        input_file = os.path.join(data_dir, "input.txt")
        vocab_file = os.path.join(data_dir, "vocab.pkl")
        tensor_file = os.path.join(data_dir, "data.npy")

        if not (os.path.exists(vocab_file) and os.path.exists(tensor_file)):
            print("reading text file")
            self.preprocess(input_file, vocab_file, tensor_file)
        else:
            print("loading preprocessed files")
            self.load_preprocessed(vocab_file, tensor_file)
        self.create_batches()
        self.reset_batch_pointer()

    def preprocess(self, input_file, vocab_file, tensor_file):
        with codecs.open(input_file, "r", encoding=self.encoding) as f:
#        with codecs.open(input_file, 'rb') as f:
            data = f.read()
        counter = collections.Counter(data)
        count_pairs = sorted(counter.items(), key=lambda x: -x[1])
        self.chars, _ = zip(*count_pairs)
        self.vocab_size = len(self.chars)
        self.vocab = dict(zip(self.chars, range(len(self.chars))))
        with open(vocab_file, 'wb') as f:
            cPickle.dump(self.chars, f)
        self.tensor = np.array(list(map(self.vocab.get, data)))
        np.save(tensor_file, self.tensor)

    def load_preprocessed(self, vocab_file, tensor_file):
        with open(vocab_file, 'rb') as f:
            self.chars = cPickle.load(f)
        self.vocab_size = len(self.chars)
        self.vocab = dict(zip(self.chars, range(len(self.chars))))
        self.tensor = np.load(tensor_file)
        self.num_batches = int(self.tensor.size / (self.batch_size *
                                                   self.seq_length))

    def create_batches(self):
        self.num_batches = int(self.tensor.size / (self.batch_size *
                                                   self.seq_length))

        # When the data (tensor) is too small,
        # let's give them a better error message
        if self.num_batches == 0:
            assert False, "Not enough data. Make seq_length and batch_size small."

        self.tensor = self.tensor[:self.num_batches * self.batch_size * self.seq_length]
        xdata = self.tensor
        ydata = np.copy(self.tensor)
        ydata[:-1] = xdata[1:]
        ydata[-1] = xdata[0]
        self.x_batches = np.split(xdata.reshape(self.batch_size, -1),
                                  self.num_batches, 1)
        self.y_batches = np.split(ydata.reshape(self.batch_size, -1),
                                  self.num_batches, 1)

    def next_batch(self):
        x, y = self.x_batches[self.pointer], self.y_batches[self.pointer]
        self.pointer += 1
        return x, y

    def reset_batch_pointer(self):
        self.pointer = 0

In [2]:
# -*- coding: utf-8 -*-
# Char-RNN 예제
# Author : solaris33
# Project URL : http://solarisailab.com/archives/2487
# GitHub Repository : https://github.com/solaris33/char-rnn-tensorflow/
# Reference : https://github.com/sherjilozair/char-rnn-tensorflow

import tensorflow as tf
import numpy as np
#from utils import TextLoader

# 학습에 필요한 설정값들을 지정합니다.
#data_dir = 'data/tinyshakespeare' # 셰익스피어 희곡 <리처드 3세> 데이터로 학습
data_dir = 'data/linux' # <Linux 소스코드> 데이터로 학습
batch_size = 50 # Training : 50, Sampling : 1
seq_length = 50 # Training : 50, Sampling : 1
hidden_size = 128   # 히든 레이어의 노드 개수
learning_rate = 0.002
num_epochs = 2
num_hidden_layers = 2
grad_clip = 5   # Gradient Clipping에 사용할 임계값

# TextLoader를 이용해서 데이터를 불러옵니다.
data_loader = TextLoader(data_dir, batch_size, seq_length)
# 학습데이터에 포함된 모든 단어들을 나타내는 변수인 chars와 chars에 id를 부여해 dict 형태로 만든 vocab을 선언합니다.
chars = data_loader.chars 
vocab = data_loader.vocab
vocab_size = data_loader.vocab_size # 전체 단어개수

# 인풋데이터와 타겟데이터, 배치 사이즈를 입력받기 위한 플레이스홀더를 설정합니다.
input_data = tf.placeholder(tf.int32, shape=[None, None])  # input_data : [batch_size, seq_length])
target_data = tf.placeholder(tf.int32, shape=[None, None]) # target_data : [batch_size, seq_length])
state_batch_size = tf.placeholder(tf.int32, shape=[])      # Training : 50, Sampling : 1

# RNN의 마지막 히든레이어의 출력을 소프트맥스 출력값으로 변환해주기 위한 변수들을 선언합니다.
# hidden_size -> vocab_size
softmax_w = tf.Variable(tf.random_normal(shape=[hidden_size, vocab_size]), dtype=tf.float32)
softmax_b = tf.Variable(tf.random_normal(shape=[vocab_size]), dtype=tf.float32)

# num_hidden_layers만큼 LSTM cell(히든레이어)를 선언합니다.
cells = []
for _ in range(0, num_hidden_layers):
    cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_size)
    cells.append(cell)

# cell을 종합해서 RNN을 정의합니다.
cell = tf.contrib.rnn.MultiRNNCell(cells, state_is_tuple=True)

# 인풋데이터를 변환하기 위한 Embedding Matrix를 선언합니다.
# vocab_size -> hidden_size
embedding = tf.Variable(tf.random_normal(shape=[vocab_size, hidden_size]), dtype=tf.float32)
inputs = tf.nn.embedding_lookup(embedding, input_data)

# 초기 state 값을 0으로 초기화합니다.
initial_state = cell.zero_state(state_batch_size, tf.float32)

# 학습을 위한 tf.nn.dynamic_rnn을 선언합니다.
# outputs : [batch_size, seq_length, hidden_size]
outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, initial_state=initial_state, dtype=tf.float32)
# ouputs을 [batch_size * seq_length, hidden_size]] 형태로 바꿉니다.
output = tf.reshape(outputs, [-1, hidden_size])

# 최종 출력값을 설정합니다.
# logits : [batch_size * seq_length, vocab_size]
logits = tf.matmul(output, softmax_w) + softmax_b
probs = tf.nn.softmax(logits)

# Cross Entropy 손실 함수를 정의합니다. 
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=target_data))

# 옵티마이저를 선언하고 옵티마이저에 Gradient Clipping을 적용합니다.
# grad_clip(=5)보다 큰 Gradient를 5로 Clippin합니다.
tvars = tf.trainable_variables()
grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
optimizer = tf.train.AdamOptimizer(learning_rate)
train_step = optimizer.apply_gradients(zip(grads, tvars))

# 세션을 열고 학습을 진행합니다.
with tf.Session() as sess:
    # 변수들에 초기값을 할당합니다.
    sess.run(tf.global_variables_initializer())
    
    for e in range(num_epochs):
        data_loader.reset_batch_pointer()
        # 초기 상태값을 지정합니다.
        state = sess.run(initial_state, feed_dict={state_batch_size : batch_size})

        for b in range(data_loader.num_batches):
            # x, y 데이터를 불러옵니다.
            x, y = data_loader.next_batch()
            # y에 one_hot 인코딩을 적용합니다. 
            y = tf.one_hot(y, vocab_size)            # y : [batch_size, seq_length, vocab_size]
            y = tf.reshape(y, [-1, vocab_size])       # y : [batch_size * seq_length, vocab_size]
            y = y.eval()

            # feed-dict에 사용할 값들과 LSTM 초기 cell state(feed_dict[c])값과 hidden layer 출력값(feed_dict[h])을 지정합니다.
            feed_dict = {input_data : x, target_data: y, state_batch_size : batch_size}
            for i, (c, h) in enumerate(initial_state):
                feed_dict[c] = state[i].c
                feed_dict[h] = state[i].h

            # 한스텝 학습을 진행합니다.
            _, loss_print, state = sess.run([train_step, loss, final_state], feed_dict=feed_dict)

            print("{}(학습한 배치개수)/{}(학습할 배치개수), 반복(epoch): {}, 손실함수(loss): {:.3f}".format(
                          e * data_loader.num_batches + b,
                          num_epochs * data_loader.num_batches,
                          (e+1), 
                          loss_print))

    print("트레이닝이 끝났습니다!")   
    

    # 샘플링 시작
    print("샘플링을 시작합니다!")
    num_sampling = 4000  # 생성할 글자(Character)의 개수를 지정합니다. 
    prime = u' '         # 시작 글자를 ' '(공백)으로 지정합니다.
    sampling_type = 1    # 샘플링 타입을 설정합니다.
    state = sess.run(cell.zero_state(1, tf.float32)) # RNN의 최초 state값을 0으로 초기화합니다.

    # Random Sampling을 위한 weighted_pick 함수를 정의합니다.
    def weighted_pick(weights):
        t = np.cumsum(weights)
        s = np.sum(weights)
        return(int(np.searchsorted(t, np.random.rand(1)*s)))

    ret = prime       # 샘플링 결과를 리턴받을 ret 변수에 첫번째 글자를 할당합니다.
    char = prime[-1]   # Char-RNN의 첫번쨰 인풋을 지정합니다.  
    for n in range(num_sampling):
        x = np.zeros((1, 1))
        x[0, 0] = vocab[char]

        # RNN을 한스텝 실행하고 Softmax 행렬을 리턴으로 받습니다.
        feed_dict = {input_data: x, state_batch_size : 1, initial_state: state}
        [probs_result, state] = sess.run([probs, final_state], feed_dict=feed_dict)         

        # 불필요한 차원을 제거합니다.
        # probs_result : (1,65) -> p : (65)
        p = np.squeeze(probs_result)

        # 샘플링 타입에 따라 3가지 종류로 샘플링 합니다.
        # sampling_type : 0 -> 다음 글자를 예측할때 항상 argmax를 사용
        # sampling_type : 1(defualt) -> 다음 글자를 예측할때 항상 random sampling을 사용
        # sampling_type : 2 -> 다음 글자를 예측할때 이전 글자가 ' '(공백)이면 random sampling, 그렇지 않을 경우 argmax를 사용
        if sampling_type == 0:
            sample = np.argmax(p)
        elif sampling_type == 2:
            if char == ' ':
                sample = weighted_pick(p)
            else:
                sample = np.argmax(p)
        else:
            sample = weighted_pick(p)

        pred = chars[sample]
        ret += pred     # 샘플링 결과에 현재 스텝에서 예측한 글자를 추가합니다. (예를들어 pred=L일 경우, ret = HEL -> HELL)
        char = pred     # 예측한 글자를 다음 RNN의 인풋으로 사용합니다.

    print("샘플링 결과:")
    print(ret)
    
    # tensorboard --logdir=./logs --port=any number you want
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter('./logs', sess.graph)

C:\Users\joobong\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


reading text file
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

0(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 5.060
1(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 4.553
2(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 4.244
3(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 4.132
4(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.907
5(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.759
6(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.682
7(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.514
8(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.508
9(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.474
10(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.464
11(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.456
12(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.411
13(학습한 배치개

128(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 2.044
129(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.997
130(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.940
131(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.958
132(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.886
133(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.973
134(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.966
135(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.929
136(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.973
137(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.963
138(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 2.041
139(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.984
140(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.998
141(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.981
142(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.955
143(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.960
144(학습한 

261(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.727
262(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.781
263(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.813
264(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.838
265(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.772
266(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.690
267(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.607
268(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.763
269(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.800
270(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.704
271(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.698
272(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.671
273(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.594
274(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.814
275(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.618
276(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.730
277(학습한 

394(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.619
395(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.669
396(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.693
397(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.660
398(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.703
399(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.631
400(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.678
401(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.655
402(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.533
403(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.562
404(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.620
405(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.550
406(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.785
407(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.753
408(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.694
409(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.720
410(학습한 

527(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.469
528(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.463
529(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.576
530(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.549
531(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.553
532(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.396
533(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.454
534(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.558
535(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.519
536(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.505
537(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.569
538(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.468
539(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.417
540(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.516
541(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.481
542(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.404
543(학습한 

660(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.419
661(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.498
662(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.364
663(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.393
664(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.324
665(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.361
666(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.476
667(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.514
668(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.412
669(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.391
670(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.317
671(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.349
672(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.467
673(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.401
674(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.403
675(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.370
676(학습한 

793(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.328
794(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.429
795(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.373
796(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.415
797(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.248
798(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.295
799(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.307
800(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.341
801(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.361
802(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.371
803(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.432
804(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.410
805(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.313
806(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.497
807(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.374
808(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.293
809(학습한 

926(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.365
927(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.422
928(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.355
929(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.318
930(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.323
931(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.342
932(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.313
933(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.323
934(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.320
935(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.413
936(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.285
937(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.264
938(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.311
939(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.345
940(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.393
941(학습한 배치개수)/1002(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.384
942(학습한 